In [4]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
from scipy.optimize import minimize
from utility_code.utility import utils

In [58]:
#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")

supp = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')

supp = supp[supp['Location'].isin(train['Province_State'])]
supp['Population'] = supp['Number of COVID-19 Cases'].divide(supp['COVID-19 Cases per 1,000,000 Population']) * 1e6

states = pd.Series.unique(train['Province_State'])
num_states = len(states)

In [51]:
state = 'California'
a = statesdata[state]

#only look at the features in features list
features = ['Confirmed','Deaths']
num_features = len(features)
WINDOW_SIZE = 5
state_feature_indices = utils.get_column_indices(statesdata['California'],features)

#stratify by state (into state dictionary)
statesdata = {}
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s,features]

#append the feature spaces from the W days prior (where W is the window length)
new_features = []
for day in range(WINDOW_SIZE):
    for f in features:
        new_features.append(f + "(-"+ str(WINDOW_SIZE-day) + " days)")
all_new_features = new_features + features


In [171]:
from itertools import chain

#fill the knn data using days from training set
knndata = pd.DataFrame(columns = all_new_features)
num_training_days = len(statesdata['California'])

#fill the table
for d in range(WINDOW_SIZE,num_training_days):
    knndata_row_index = knndata.shape[0]
    knn_row = utils.flatten_dataframe(a,slice(d-WINDOW_SIZE,d+1), state_feature_indices)
    utils.dataframe_append_row(knndata,knn_row,state,d)     

In [157]:
knndata

,Confirmed(-5 days),Deaths(-5 days),Confirmed(-4 days),Deaths(-4 days),Confirmed(-3 days),Deaths(-3 days),Confirmed(-2 days),Deaths(-2 days),Confirmed(-1 days),Deaths(-1 days),Confirmed,Deaths
California (day 5),22795,640,23931,714,25356,767,26686,860,27677,956,29157,1037
California (day 6),23931,714,25356,767,26686,860,27677,956,29157,1037,30491,1140
California (day 7),25356,767,26686,860,27677,956,29157,1037,30491,1140,31431,1177
California (day 8),26686,860,27677,956,29157,1037,30491,1140,31431,1177,33686,1225
California (day 9),27677,956,29157,1037,30491,1140,31431,1177,33686,1225,35465,1282
...,...,...,...,...,...,...,...,...,...,...,...,...
California (day 137),664435,12131,669280,12153,675561,12235,681032,12382,687004,12532,691821,12677
California (day 138),669280,12153,675561,12235,681032,12382,687004,12532,691821,12677,697385,12805
California (day 139),675561,12235,681032,12382,687004,12532,691821,12677,697385,12805,702038,12894
California (day 140),681032,12382,687004,12532,691821,12677,697385,12805,702038,12894,705951,12937


In [158]:
x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
y = knndata[['Confirmed', 'Deaths']]

In [159]:
toguess = 1
trainx = x.head(len(x)-toguess)
testx = x.tail(toguess)
trainy = y.head(len(y)-toguess)
testy = y.tail(toguess)

In [160]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
reg = MultiOutputRegressor(estimator=Ridge(random_state=123))
reg.fit(trainx, trainy)
reg.predict(testx)

array([[712739.38560304,  13059.79652614]])

In [172]:
states

days_to_predict = 26
for d in range(days_to_predict):
    x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
    y = knndata[['Confirmed', 'Deaths']]
    toguess = 1
    trainx = x.head(len(x))
    trainy = y.head(len(y))
    # testy = y.tail(toguess)

    reg.fit(trainx, trainy)
    

    ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
    #ftrs.drop(columns=knndata.columns[[0,1]])
   
    ftrs.columns = knndata.columns[0:10]
    
    new = reg.predict(ftrs)
    ftrs = np.append(ftrs, new)
    ftrs = ftrs.astype(int)
    if d==0: print(ftrs)
    knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)
    
    # append to knndata
    if d == 0: 
        print(knndata)
done = knndata.tail(days_to_predict)
done = done[['Confirmed', 'Deaths']]
print(done)

[691821  12677 697385  12805 702038  12894 705951  12937 712475  13022
 720306  13170]
    Confirmed(-5 days) Deaths(-5 days) Confirmed(-4 days) Deaths(-4 days)  \
0                22795             640              23931             714   
1                23931             714              25356             767   
2                25356             767              26686             860   
3                26686             860              27677             956   
4                27677             956              29157            1037   
..                 ...             ...                ...             ...   
133             669280           12153             675561           12235   
134             675561           12235             681032           12382   
135             681032           12382             687004           12532   
136             687004           12532             691821           12677   
137             691821           12677             697385         

In [162]:
knndata

,Confirmed(-5 days),Deaths(-5 days),Confirmed(-4 days),Deaths(-4 days),Confirmed(-3 days),Deaths(-3 days),Confirmed(-2 days),Deaths(-2 days),Confirmed(-1 days),Deaths(-1 days),Confirmed,Deaths
0,22795,640,23931,714,25356,767,26686,860,27677,956,29157,1037
1,23931,714,25356,767,26686,860,27677,956,29157,1037,30491,1140
2,25356,767,26686,860,27677,956,29157,1037,30491,1140,31431,1177
3,26686,860,27677,956,29157,1037,30491,1140,31431,1177,33686,1225
4,27677,956,29157,1037,30491,1140,31431,1177,33686,1225,35465,1282
...,...,...,...,...,...,...,...,...,...,...,...,...
158,849240,15333,858182,15469,867269,15606,876543,15747,885952,15891,895453,16036
159,858182,15469,867269,15606,876543,15747,885952,15891,895453,16036,905003,16181
160,867269,15606,876543,15747,885952,15891,895453,16036,905003,16181,914609,16325
161,876543,15747,885952,15891,895453,16036,905003,16181,914609,16325,924318,16470


In [273]:
proj = {}
for s in states:
    
    a = statesdata[s]

    #fill the knn data using days from training set
    knndata = pd.DataFrame(columns = all_new_features)
    num_training_days = len(statesdata['California'])

    #fill the table
    for d in range(WINDOW_SIZE,num_training_days):
        knndata_row_index = knndata.shape[0]
        knn_row = utils.flatten_dataframe(a,slice(d-WINDOW_SIZE,d+1), state_feature_indices)
        utils.dataframe_append_row(knndata,knn_row,state,d)   
    

    days_to_predict = 26
    for d in range(days_to_predict):
        x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
        y = knndata[['Confirmed', 'Deaths']]
        toguess = 1
        trainx = x.head(len(x))
        trainy = y.head(len(y))
        # testy = y.tail(toguess)

        reg.fit(trainx, trainy)


        ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
        #ftrs.drop(columns=knndata.columns[[0,1]])

        ftrs.columns = knndata.columns[0:10]

        new = reg.predict(ftrs)
        ftrs = np.append(ftrs, new)
        ftrs = ftrs.astype(int)
        #if d==0: print(ftrs)
        knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)

        # append to knndata
        #if d == 0: 
            #print(knndata)
    done = knndata.tail(days_to_predict)
    done = done[['Confirmed', 'Deaths']]
    #print(done)
    proj[s] = done

In [275]:
statesdata['Alaska']

,Confirmed,Deaths
1,272,8
51,277,8
101,285,9
151,293,9
201,300,9
...,...,...
6851,4974,37
6901,5092,37
6951,5182,37
7001,5226,37


In [281]:
order = test.loc[0:49,'Province_State']
test

# format submission
conf = []
dead = []
fid = 0
print(sub)
for i in range(days_to_predict):
    for j in order:
        projection = proj[j].iloc[i]
        print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        dead.append(int(projection['Deaths']))
        print(fid)
        fid+=1
    
    

Empty DataFrame
Columns: [ForecastID, Province_State, Date, Confirmed, Deaths]
Index: []
Alabama day 0
0
Alaska day 0
1
Arizona day 0
2
Arkansas day 0
3
California day 0
4
Colorado day 0
5
Connecticut day 0
6
Delaware day 0
7
Florida day 0
8
Georgia day 0
9
Hawaii day 0
10
Idaho day 0
11
Illinois day 0
12
Indiana day 0
13
Iowa day 0
14
Kansas day 0
15
Kentucky day 0
16
Louisiana day 0
17
Maine day 0
18
Maryland day 0
19
Massachusetts day 0
20
Michigan day 0
21
Minnesota day 0
22
Mississippi day 0
23
Missouri day 0
24
Montana day 0
25
Nebraska day 0
26
Nevada day 0
27
New Hampshire day 0
28
New Jersey day 0
29
New Mexico day 0
30
New York day 0
31
North Carolina day 0
32
North Dakota day 0
33
Ohio day 0
34
Oklahoma day 0
35
Oregon day 0
36
Pennsylvania day 0
37
Rhode Island day 0
38
South Carolina day 0
39
South Dakota day 0
40
Tennessee day 0
41
Texas day 0
42
Utah day 0
43
Vermont day 0
44
Virginia day 0
45
Washington day 0
46
West Virginia day 0
47
Wisconsin day 0
48
Wyoming day 0
49

North Carolina day 9
482
North Dakota day 9
483
Ohio day 9
484
Oklahoma day 9
485
Oregon day 9
486
Pennsylvania day 9
487
Rhode Island day 9
488
South Carolina day 9
489
South Dakota day 9
490
Tennessee day 9
491
Texas day 9
492
Utah day 9
493
Vermont day 9
494
Virginia day 9
495
Washington day 9
496
West Virginia day 9
497
Wisconsin day 9
498
Wyoming day 9
499
Alabama day 10
500
Alaska day 10
501
Arizona day 10
502
Arkansas day 10
503
California day 10
504
Colorado day 10
505
Connecticut day 10
506
Delaware day 10
507
Florida day 10
508
Georgia day 10
509
Hawaii day 10
510
Idaho day 10
511
Illinois day 10
512
Indiana day 10
513
Iowa day 10
514
Kansas day 10
515
Kentucky day 10
516
Louisiana day 10
517
Maine day 10
518
Maryland day 10
519
Massachusetts day 10
520
Michigan day 10
521
Minnesota day 10
522
Mississippi day 10
523
Missouri day 10
524
Montana day 10
525
Nebraska day 10
526
Nevada day 10
527
New Hampshire day 10
528
New Jersey day 10
529
New Mexico day 10
530
New York day 10


Florida day 20
1008
Georgia day 20
1009
Hawaii day 20
1010
Idaho day 20
1011
Illinois day 20
1012
Indiana day 20
1013
Iowa day 20
1014
Kansas day 20
1015
Kentucky day 20
1016
Louisiana day 20
1017
Maine day 20
1018
Maryland day 20
1019
Massachusetts day 20
1020
Michigan day 20
1021
Minnesota day 20
1022
Mississippi day 20
1023
Missouri day 20
1024
Montana day 20
1025
Nebraska day 20
1026
Nevada day 20
1027
New Hampshire day 20
1028
New Jersey day 20
1029
New Mexico day 20
1030
New York day 20
1031
North Carolina day 20
1032
North Dakota day 20
1033
Ohio day 20
1034
Oklahoma day 20
1035
Oregon day 20
1036
Pennsylvania day 20
1037
Rhode Island day 20
1038
South Carolina day 20
1039
South Dakota day 20
1040
Tennessee day 20
1041
Texas day 20
1042
Utah day 20
1043
Vermont day 20
1044
Virginia day 20
1045
Washington day 20
1046
West Virginia day 20
1047
Wisconsin day 20
1048
Wyoming day 20
1049
Alabama day 21
1050
Alaska day 21
1051
Arizona day 21
1052
Arkansas day 21
1053
California day 21

In [283]:
test['Confirmed'] = conf
test['Deaths'] = dead

In [272]:
len(conf)

1107

In [259]:
len(test)

1300

In [295]:
submission = test.drop(columns=['Date', 'Province_State'])
print(submission)
submission.to_csv("team31-nov29.csv", index=False, header=True)

      ForecastID  Confirmed  Deaths
0              0     127250    2193
1              1       5325      36
2              2     202159    5061
3              3      61738     810
4              4     720306   13170
...          ...        ...     ...
1295        1295     147052    2868
1296        1296      90375    2181
1297        1297      15088     268
1298        1298      97875    1285
1299        1299       5191      37

[1300 rows x 3 columns]


In [285]:
proj['Arkansas']

,Confirmed,Deaths
137,61738,810
138,62217,824
139,62721,838
140,63231,852
141,63750,866
142,64267,880
143,64785,894
144,65302,908
145,65817,922
146,66329,936


In [288]:
test

,ForecastID,Province_State,Date,Confirmed,Deaths
0,0,Alabama,09-01-2020,127250,2193
1,1,Alaska,09-01-2020,5325,36
2,2,Arizona,09-01-2020,202159,5061
3,3,Arkansas,09-01-2020,61738,810
4,4,California,09-01-2020,720306,13170
...,...,...,...,...,...
1295,1295,Virginia,09-26-2020,147052,2868
1296,1296,Washington,09-26-2020,90375,2181
1297,1297,West Virginia,09-26-2020,15088,268
1298,1298,Wisconsin,09-26-2020,97875,1285


,ForecastID,Province_State,Date,Confirmed,Deaths
0,0,Alabama,09-01-2020,127250,2193
1,1,Alaska,09-01-2020,5325,36
2,2,Arizona,09-01-2020,202159,5061
3,3,Arkansas,09-01-2020,61738,810
4,4,California,09-01-2020,720306,13170
...,...,...,...,...,...
1295,1295,Virginia,09-26-2020,147052,2868
1296,1296,Washington,09-26-2020,90375,2181
1297,1297,West Virginia,09-26-2020,15088,268
1298,1298,Wisconsin,09-26-2020,97875,1285


Index(['ForecastID', 'Province_State', 'Date', 'Confirmed', 'Deaths'], dtype='object')